In [1]:
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import mappings

# Set the theme once
pio.templates.default = "plotly_white"

from util import multi_select_counts, get_subplot_coords

df_pre = pl.read_csv("OriginalData/Preprogram_Survey.csv")
df_post = pl.read_csv("OriginalData/PostOrientation_Survey.csv")

In [2]:
df_post

#,How confident are you now in managing your emotions and staying calm when things get stressful at work or in life?,"After learning about emotional intelligence, how are you at recognizing your personal emotional triggers?",Understanding common workplace triggers,Seeing the difference between low vs. high emotional intelligence,Creating my Emotional Pause Plan,Learning ways to stay calm and in control,Talking with others about emotional stress,Visualizing myself responding better in tough situations,Identifying how others see me when I’m stressed,Other,"After the Communication Styles workshop, *how well do you understand your own communication style?*",What’s the communication habit _*you most want to work on this summer?*_,Other_duplicated_0,Are you more or less motivated about setting and actually reaching your goals after the goals workshop?,What is *ONE specific personal or professional goal* you plan to work on this summer?,What is your current clarity level about your possible career paths?,Exploring what I’m naturally good at,Seeing how hobbies or skills connect to real careers,Talking with others about dream jobs and paths,Identifying what I don’t want in a job,Learning that career paths can change over time,Getting new ideas I never considered before,"Realizing I don’t need a full plan, just a direction",Other_duplicated_1,How confident are you in your ability *to manage your money* this summer?,Which lessons from the *Money Moves workshop *do you think will help you most? *(Select TWO)*,Other_duplicated_2,"After the personal branding session, how confident are you in *how you show up professionally*?",Practice my elevator pitch,Start or improve a professional social media profile,Ask for feedback about how I come across to others,Be more intentional about how I speak and carry myself,Make a better first impression in new spaces,Network and build relationships,Post content that shows who I am professionally,Other_duplicated_3,"Overall, how much do you feel you grew during the orientation?",Would you recommend this orientation and program to a friend or peer?,What are *TWO topics or focus areas* you wish we included in orientation that weren’t covered?,What’s *ONE SUGGESTION* you have for how we can make orientation even better next time?,What were *TWO THINGS* about this orientation that you enjoyed or found most valuable?,How old are you?,What is your current education status?,Other_duplicated_4,What is your gender identity?,Other_duplicated_5,Black or African American,Hispanic or Latino,White,Asian or Pacific Islander,American Indian or Alaska Native,"Have you signed up to receive your paycheck with """"Direct Deposit"""" and have a bank account?","If you are set up for """"direct deposit"""", what is the name of your bank?",Response Type,Start Date (UTC),Stage Date (UTC),Submit Date (UTC),Network ID,Tags,Ending
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""3r116t1hh4jn7wq63r116toyehnbqx…","""Extremely confident ""","""A lot better, I understand the…",null,null,null,"""Learning ways to stay calm and…",null,"""Visualizing myself responding …",null,null,"""I understand it clearly ""","""Knowing how to adjust dependin…",null,"""Much more motivated ""","""Working out""","""I have solid ideas ""","""Exploring what I’m naturally g…",null,null,null,null,null,"""Realizing I don’t need a full …",null,"""Extremely confident ""","""Prioritizing needs vs. wants """,null,"""Very confident """,null,null,null,"""Be more intentional about how …",null,"""Network and build relationship…",null,null,"""A lot ""","""Absolutely yes ""","""Everything was good""","""Food Food Food 9amto2pm no foo…","""The people and the atmosphere …","""14-17""","""In high school """,null,"""Male """,null,"""Black or African American ""","""Hispanic or Latin

In [3]:
df_growth = df_post.select(["#"] + mappings.growth_questions).to_pandas()

In [4]:
df_growth = pl.DataFrame(df_growth.replace(mappings.all_post_orientation_entry_mappings).rename(columns=mappings.original_to_concise_question_names))

# Define the logical order for each set of responses
# These lists should contain the *standardized* entries in your desired order
order_confidence_emotions = ['Highly Confident', 'Very Confident', 'Slightly Confident', 'Not Confident At All']
order_emotional_triggers = ['Significantly Better', 'Much Better', 'Slightly Better', 'Not Better At All']
order_communication_style = ['Clearly Understand', 'Understand & Improving', 'General Understanding', 'Do Not Understand']
order_goal_motivation = ['Highly Motivated', 'Much More Motivated', 'Slightly More Motivated', 'No Change', 'Less Motivated']
order_career_paths = ['Very Clear', 'Solid Ideas', 'Slightly Clearer', 'Still Unsure']
order_money_management = ['Highly Confident', 'Very Confident', 'Somewhat Confident', 'Not Confident At All'] # Duplicate of confidence, but good to keep explicit
order_professional_presence = ['Highly Confident', 'Very Confident', 'Somewhat Confident', 'Slightly More Confident', 'Not Yet Confident']
order_overall_growth = ['More Than Expected', 'Significant Growth', 'Some Growth', 'Slight Growth', 'No Growth At All']

# A dictionary to map question names (or a unique identifier for them) to their respective orders
post_orientation_question_orders = {
    "Confidence in Managing Emotions": order_confidence_emotions,
    "Recognizing Emotional Triggers": order_emotional_triggers,
    "Understanding Communication Style": order_communication_style,
    "Motivation for Goal Setting": order_goal_motivation,
    "Clarity on Career Paths": order_career_paths,
    "Confidence in Money Management": order_money_management,
    "Confidence in Professional Presence": order_professional_presence,
    "Overall Growth": order_overall_growth
}

In [5]:
fig = make_subplots(rows=2, cols=4,
                    subplot_titles=list(post_orientation_question_orders.keys()))

for index, g in enumerate(df_growth.columns[1:]):
    
    position = get_subplot_coords(index, 2, 4)
    
    d = multi_select_counts(df_growth, ["#", g])
    fig.add_trace(
        go.Bar(y=d.values, x=d.index),
        row=position[0], col=position[1])
    
    fig.update_xaxes(
        categoryorder='array', # Ensure Plotly respects the order of the array
        categoryarray=post_orientation_question_orders[g], # Pass the order directly to Plotly for good measure
        tickangle=-45, # Adjust as needed for label readability
        row=position[0], col=position[1]
    )

In [6]:
# Update layout
fig.update_layout(height=1000, width=1800, title_text="")
fig.update_layout(showlegend=False)
"""fig.update_xaxes(
    categoryorder='array',
    categoryarray=['Not at all', 'Slightly', 'Moderately','Very', 'Extremely'])"""
fig.show()

In [7]:
df_growth

#,Confidence in Managing Emotions,Recognizing Emotional Triggers,Understanding Communication Style,Motivation for Goal Setting,Clarity on Career Paths,Confidence in Money Management,Confidence in Professional Presence,Overall Growth
str,str,str,str,str,str,str,str,str
"""3r116t1hh4jn7wq63r116toyehnbqx…","""Highly Confident""","""Significantly Better""","""Clearly Understand""","""Much More Motivated""","""Solid Ideas""","""Highly Confident""","""Very Confident""","""Significant Growth"""
"""5pw9w5nvzrtf5pw977xk00jo2vl30v…","""Slightly Confident""","""Much Better""","""General Understanding""","""Much More Motivated""","""Solid Ideas""","""Very Confident""","""Somewhat Confident""","""Significant Growth"""
"""8phhspzlh9jx7h8phhjukj0rddkoxl…","""Slightly Confident""","""Much Better""","""General Understanding""","""Much More Motivated""","""Solid Ideas""","""Somewhat Confident""","""Very Confident""","""Significant Growth"""
"""k26pi6bicbsvzd2yfk26pi67a2u4fh…","""Highly Confident""","""Significantly Better""","""Clearly Understand""","""Slightly More Motivated""","""Slightly Clearer""","""Very Confident""","""Very Confident""","""Some Growth"""
"""6pb6dn5vxn2itktj6pb6eidta6hte2…","""Highly Confident""","""Significantly Better""","""Clearly Understand""","""Much More Motivated""","""Solid Ideas""","""Highly Confident""","""Very Confident""","""More Than Expected"""
…,…,…,…,…,…,…,…,…
"""2erc4226su0ht44yld522erc42gyew…","""Very Confident""","""Much Better""","""General Understanding""","""Slightly More Motivated""","""Solid Ideas""","""Highly Confident""","""Slightly More Confident""","""Some Growth"""
"""je289s3qu2nabiqhkg2m7g2je28lpy…","""Very Confident""","""Much Better""","""General Understanding""","""Slightly More Motivated""","""Solid Ideas""","""Somewhat Confident""","""Very Confident""","""More Than Expected"""
"""oq7lvo5xhpvpcdv2toq7lvhz221pid…","""Very Confident""","""Slightly Better""","""General Understanding""","""Much More Motivated""","""Slightly Clearer""","""Very Confident""","""Somewhat Confident""","""Some Growth"""


In [8]:
df_numeric = df_growth.drop(["#"]).to_pandas().replace(mappings.all_post_orientation_score_map)

# Step 3: Average across all responses per skill
avg_scores = df_numeric.mean() + 1

# Step 4: Plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=avg_scores.tolist(),
    theta=avg_scores.index.tolist(),
    fill='toself',
    name='Average Scores'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[1, 5],
            tickvals=[1, 5],  
            ticktext=["Low", "High"]  
        )
    ),
    title="Post Orientation Self-Assessment Across Key Skills",
    showlegend=False
)

fig.show()

/tmp/ipykernel_30362/2021950361.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [9]:
post_orientation_mapping = {
    "How confident are you now in managing your emotions and staying calm when things get stressful at work or in life?": [
        "I can manage challenges without giving up"
    ],
    "After learning about emotional intelligence, how are you at recognizing your personal emotional triggers?": [
        "Stressed or overwhelmed"
    ],
    "After the Communication Styles workshop, how well do you understand your own communication style?": [
        "Communicating professionally (verbal and written)"
    ],
    "Are you more or less motivated about setting and actually reaching your goals after the goals workshop?": [
        "Motivated to work toward your goals",
        "Focused and productive"
    ],
    "What is your current clarity level about your possible career paths?": [
        "Do you have a *career plan or clear next steps?*"
    ],
    "How confident are you in your ability *to manage your money* this summer?": [
        "Creating a personal budget",
        "Understanding credit and credit scores",
    ],
    "After the personal branding session, how confident are you in *how you show up professionally*?": [
        "I know how to present myself professionally in different settings"
    ],
    "Overall, how much do you feel you grew during the orientation?": []
}

all_matched_columns = [col for cols in post_orientation_mapping.values() for col in cols]


In [10]:
df_pre_comp = df_pre.select(all_matched_columns).to_pandas()

In [11]:
response_mapping = {
    "I can manage challenges without giving up": {
        'Strongly disagree': 0,
        'Disagree': 1,
        'Neutral ': 2,
        'Agree': 3,
        'Strongly agree': 4,
        None: None
    },
    "Stressed or overwhelmed": {
        'Very Often': 0,
        'Often': 1,
        'Sometimes': 2,
        'Rarely': 3,
        'Never': 4,
        None: None
    },
    "Communicating professionally (verbal and written)": {
        'Not confident': 0,
        'Slightly ': 1,
        'Somewhat': 2,
        'Very': 3,
        'Extremely': 4
    },
    "Motivated to work toward your goals": {
        'Very Often': 4,
        'Often': 3,
        'Sometimes': 2,
        'Rarely': 1,
        'Never': 0,
        None: None
    },
    "Focused and productive": {
        'Very Often': 4,
        'Often': 3,
        'Sometimes': 2,
        'Rarely': 1,
        'Never': 0,
        None: None
    },
    "Do you have a *career plan or clear next steps?*": {
        "I’m still exploring and need guidance": 0,
        'I have a general idea but need help organizing it': 1,
        "Yes, I have a clear plan I’m working on": 2,
        None: None
    },
    "Creating a personal budget": {
        'Not familiar': 0,
        'Slightly': 1,
        'Somewhat': 2,
        'Very': 3,
        'Extremely': 4
    },
    "Understanding credit and credit scores": {
        'Not familiar': 0,
        'Slightly': 1,
        'Somewhat': 2,
        'Very': 3,
        'Extremely': 4
    },
    "I know how to present myself professionally in different settings": {
        'Strongly disagree': 0,
        'Neutral ': 1,
        'Agree': 2,
        'Strongly agree': 3,
        None: None
    }
}


In [12]:
for column, mapping in response_mapping.items():
    if column in df_pre_comp.columns:
        df_pre_comp[column] = df_pre_comp[column].replace(mapping)

/tmp/ipykernel_30362/2928776004.py:3: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [13]:
competency_to_item = {
    'Confidence in Managing Emotions': 'I can manage challenges without giving up',
    'Recognizing Emotional Triggers': 'Stressed or overwhelmed',
    'Understanding Communication Style': 'Communicating professionally (verbal and written)',
    'Motivation for Goal Setting': 'Motivated to work toward your goals',
    'Clarity on Career Paths': 'Do you have a *career plan or clear next steps?*',
    'Confidence in Money Management': 'Creating a personal budget',
    'Confidence in Professional Presence': 'I know how to present myself professionally in different settings'
}

In [14]:
reversed_dict = {v: k for k, v in competency_to_item.items()}

pre_std = pl.DataFrame(df_pre_comp).select(competency_to_item.values()).rename(reversed_dict)

In [15]:
pre_std = pre_std.unpivot(pre_std.columns, variable_name = "Competency", value_name = "value").group_by("Competency").mean().with_columns( pl.lit("Pre-Orientation"))

In [16]:
pre_std

Competency,value,literal
str,f64,str
"""Confidence in Money Management""",2.56,"""Pre-Orientation"""
"""Clarity on Career Paths""",1.171717,"""Pre-Orientation"""
"""Confidence in Managing Emotion…",3.030303,"""Pre-Orientation"""
"""Understanding Communication St…",2.7,"""Pre-Orientation"""
"""Motivation for Goal Setting""",3.151515,"""Pre-Orientation"""
"""Recognizing Emotional Triggers""",1.606061,"""Pre-Orientation"""
"""Confidence in Professional Pre…",2.252525,"""Pre-Orientation"""


In [17]:
post_std = pl.DataFrame(df_numeric).unpivot(df_numeric.columns, variable_name = "Competency").group_by("Competency").mean().with_columns( pl.lit("Post-Orientation")).filter(pl.col("Competency").is_in(pre_std["Competency"]))

In [18]:
df_comp = pl.concat([pre_std, post_std]).rename({"literal": "When"}).filter(
    pl.col("Competency") != "Confidence in Managing Emotions",
    pl.col("Competency") != "Motivation for Goal Setting"
)

In [19]:
df_comp = df_comp.with_columns(pl.col("value") + 1)

In [20]:
import plotly.express as px

# Create grouped bar chart
fig = px.bar(
    df_comp,
    x="Competency",
    y="value",
    color="When",
    barmode="group",
    labels={"value": "Average Score", "Competency": "Competency"},
    title="Pre- vs Post-Orientation Scores by Competency"
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

/usr/lib/python3/dist-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [21]:
df_post.columns

['#',
 'How confident are you now in managing your emotions and staying calm when things get stressful at work or in life?',
 'After learning about emotional intelligence, how are you at recognizing your personal emotional triggers?',
 'Understanding common workplace triggers ',
 'Seeing the difference between low vs. high emotional intelligence ',
 'Creating my Emotional Pause Plan ',
 'Learning ways to stay calm and in control ',
 'Talking with others about emotional stress ',
 'Visualizing myself responding better in tough situations ',
 'Identifying how others see me when I’m stressed ',
 'Other',
 'After the Communication Styles workshop, *how well do you understand your own communication style?*',
 'What’s the communication habit _*you most want to work on this summer?*_',
 'Other_duplicated_0',
 'Are you more or less motivated about setting and actually reaching your goals after the goals workshop?',
 'What is *ONE specific personal or professional goal* you plan to work on this

In [22]:
emo_col = ['Understanding common workplace triggers ',
 'Seeing the difference between low vs. high emotional intelligence ',
 'Creating my Emotional Pause Plan ',
 'Learning ways to stay calm and in control ',
 'Talking with others about emotional stress ',
 'Visualizing myself responding better in tough situations ',
 'Identifying how others see me when I’m stressed ']

clarity_col = ['Exploring what I’m naturally good at ',
 'Seeing how hobbies or skills connect to real careers ',
 'Talking with others about dream jobs and paths ',
 'Identifying what I don’t want in a job ',
 'Learning that career paths can change over time ',
 'Getting new ideas I never considered before ',
 'Realizing I don’t need a full plan, just a direction ']

competency_mapping = {
    # Emotional Intelligence
    'Understanding common workplace triggers ': 'Recognizing emotional triggers',
    'Seeing the difference between low vs. high emotional intelligence ': 'Understanding emotional intelligence',
    'Creating my Emotional Pause Plan ': 'Creating a pause plan',
    'Learning ways to stay calm and in control ': 'Staying calm under pressure',
    'Talking with others about emotional stress ': 'Discussing emotional stress',
    'Visualizing myself responding better in tough situations ': 'Visualizing better responses',
    'Identifying how others see me when I’m stressed ': 'Understanding how others see me when stressed',
    
    # Career Clarity
    'Exploring what I’m naturally good at ': 'Identifying strengths',
    'Seeing how hobbies or skills connect to real careers ': 'Connecting hobbies to careers',
    'Talking with others about dream jobs and paths ': 'Discussing career dreams',
    'Identifying what I don’t want in a job ': 'Knowing what I don’t want',
    'Learning that career paths can change over time ': 'Understanding career flexibility',
    'Getting new ideas I never considered before ': 'Considering new ideas',
    'Realizing I don’t need a full plan, just a direction ': 'Accepting direction over full plans'
}


In [26]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=["Emotional Intelligence and Regulation", "Career and Personal Development"])

# Previous Job Exp Bar Chart (Row 1, Col 1)
emo = multi_select_counts(df_post, ["#"] + emo_col, competency_mapping)
fig.add_trace(
    go.Bar(x=emo.index, y=emo.values),
    row=1, col=1
)

# Clarity Bar Chart (Row 1, Col 2)
clarity = multi_select_counts(df_post, ["#"] + clarity_col, competency_mapping)
fig.add_trace(
    go.Bar(x=clarity.index, y=clarity.values),
    row=1, col=2
)

# Update layout
fig.update_layout(height=600, width=1200, title_text="Areas of Identified Improvements")
fig.update_layout(showlegend=False)
fig.show( )
